# Algoritmo Simplex

Optimización Lineal, Pivotaje, Inicialización y Desarrollo Iterativo

## Resumen

Se desarrolla formalmente el algoritmo simplex como procedimiento iterativo para la resolución de problemas de programación lineal en forma estándar. Se presenta su fundamento geométrico basado en vértices de politopos convexos, las reglas de entrada y salida de la base mediante costos reducidos y análisis de razones, el concepto de pivotaje como cambio de base, los métodos de inicialización (M grande y bifásico), y el desarrollo estructurado del algoritmo en forma tabular.

# 1. Planteamiento General

Considérese el problema estándar de programación lineal:

$$
\text{Maximizar } P = c_1 x_1 + c_2 x_2 + \dots + c_n x_n
$$

Sujeto a:

$$
a_{11}x_1 + a_{12}x_2 + \dots + a_{1n}x_n \le b_1
$$
$$
a_{21}x_1 + a_{22}x_2 + \dots + a_{2n}x_n \le b_2
$$
$$
\vdots
$$
$$
a_{m1}x_1 + a_{m2}x_2 + \dots + a_{mn}x_n \le b_m
$$

$$
x_j \ge 0
$$

En forma matricial:

$$
\max c^T x
$$
$$
Ax \le b
$$
$$
x \ge 0
$$

El conjunto factible es convexo y posee un número finito de vértices.
El algoritmo simplex recorre dichos vértices de forma sistemática hasta encontrar el óptimo .

# 2. Interpretación Geométrica

Propiedades fundamentales:

* Si existe solución óptima única → es un vértice.
* Si existen múltiples soluciones óptimas → al menos dos vértices adyacentes son óptimos.
* El conjunto factible tiene número finito de vértices.
* Si un vértice mejora a sus adyacentes → es óptimo global.

El método simplex es iterativo: en cada iteración evalúa la función objetivo en un vértice adyacente .

# 3. Forma Estándar e Inicialización

Para aplicar simplex:

1. Convertir desigualdades en igualdades agregando variables de holgura:

$$
Ax + y = b
$$

2. Variables básicas iniciales: variables de holgura.
3. Variables no básicas: variables originales.
4. Función objetivo en forma:

$$
P - c_1 x_1 - \dots - c_n x_n = 0
$$

Si existen restricciones ≥ o =, se agregan variables artificiales (método M grande o bifásico) .


# 4. Tabla Inicial

Estructura:

* Primera fila: variables.
* Primera columna: variables básicas.
* Última fila: fila objetivo.
* Última columna: términos independientes.

En la tabla inicial:

* Variables básicas = variables de holgura.
* Variables no básicas = variables originales.
* P = 0.

# 5. Reglas del Algoritmo

## 5.1 Regla de Entrada

Para maximización:

$$
k = \operatorname{ArgMax}(c_j - z_j \mid c_j - z_j > 0)
$$

Se elige la variable con mayor costo reducido positivo .


## 5.2 Regla de Salida (Análisis de Razones)

$$
\theta = \min \left\{ \frac{x_{Bi}}{y_{ik}} \mid y_{ik} > 0 \right\}
$$

Si no existe $y_{ik} > 0$ → problema no acotado.

## 5.3 Prueba de Optimalidad

El proceso termina cuando no existen coeficientes negativos en la fila objetivo (maximización).

# 6. Pivotaje

El pivotaje es un cambio de base:

* Se convierte el pivote en 1.
* Se hacen ceros arriba y abajo (Gauss-Jordan).
* Se actualizan variables básicas y no básicas.

Es análogo a eliminación de Gauss, pero aplicado a sistemas con inecuaciones .

# 7. Ejemplo: Cocinetas

Maximizar:

$$
P = 5x_1 + 4x_2 + 3x_3
$$

Sujeto a:

$$
x_1 + x_2 + x_3 \le 28
$$
$$
3x_1 + 2x_2 + x_3 \le 48
$$
$$
x_i \ge 0
$$

Solución óptima:

$$
x_1 = 10,\quad x_2 = 0,\quad x_3 = 18
$$
$$
P = 104
$$


# 8. Implementación Computacional Simplificada (Simplex)

## 8.1 Construcción de tabla inicial

In [1]:
import numpy as np

def initial_simplex_tableau(A, b, c):
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    c = np.array(c, dtype=float)

    m, n = A.shape
    I = np.eye(m)

    tableau = np.zeros((m+1, n+m+1))
    tableau[:m, :n] = A
    tableau[:m, n:n+m] = I
    tableau[:m, -1] = b
    tableau[-1, :n] = -c

    return tableau

## 8.2 Paso de Pivotaje

In [2]:
def pivot(tableau, row, col):
    tableau[row] = tableau[row] / tableau[row, col]

    for r in range(len(tableau)):
        if r != row:
            tableau[r] -= tableau[r, col] * tableau[row]

    return tableau

## 8.3 Algoritmo Simplex Básico

In [3]:
def simplex(A, b, c):
    tableau = initial_simplex_tableau(A, b, c)
    m, n = tableau.shape

    while True:
        # Variable entrante
        col = np.argmin(tableau[-1, :-1])
        if tableau[-1, col] >= 0:
            break  # óptimo alcanzado

        # Análisis de razones
        ratios = []
        for i in range(m-1):
            if tableau[i, col] > 0:
                ratios.append(tableau[i, -1] / tableau[i, col])
            else:
                ratios.append(np.inf)

        row = np.argmin(ratios)

        if ratios[row] == np.inf:
            raise ValueError("Problema no acotado")

        tableau = pivot(tableau, row, col)

    return tableau

## 8.4 Ejecución del ejemplo Cocinetas

In [4]:
A = [
    [1,1,1],
    [3,2,1]
]

b = [28,48]
c = [5,4,3]

tableau_final = simplex(A, b, c)

print("Tabla final:\n", tableau_final)
print("Valor óptimo:", tableau_final[-1, -1])

Tabla final:
 [[-1.0000000e+00  0.0000000e+00  1.0000000e+00  2.0000000e+00
  -1.0000000e+00  8.0000000e+00]
 [ 2.0000000e+00  1.0000000e+00  0.0000000e+00 -1.0000000e+00
   1.0000000e+00  2.0000000e+01]
 [ 4.4408921e-16  0.0000000e+00  0.0000000e+00  2.0000000e+00
   1.0000000e+00  1.0400000e+02]]
Valor óptimo: 104.0


Resultado esperado:

$$
P = 104
$$

# 9. Condiciones Especiales

* Si no hay razón válida → problema no acotado.
* Si región factible vacía → problema no factible.
* Si múltiples costos reducidos nulos → soluciones alternativas.

# 10. Conclusiones

El algoritmo simplex:

* Es un método iterativo basado en vértices.
* Utiliza pivotaje como cambio estructural de base.
* Requiere solución factible básica inicial.
* Se fundamenta en costos reducidos.
* Termina cuando se cumple criterio de optimalidad.

Fue desarrollado por George Dantzig en los años 40 y constituye uno de los algoritmos más importantes de la optimización matemática .
